In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Kidney_Disease_Classification_DL'

In [4]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/G-Sahil123/Kidney_Disease_Classification_DL.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "G-Sahil123"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "ed797bbf811f5c2714a6f8ee4b3cc86b7d597214"


In [ ]:
# import dagshub
# dagshub.init(repo_owner='G-Sahil123', repo_name='Kidney_Disease_Classification_DL', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [11]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/G-Sahil123/Kidney_Disease_Classification_DL.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [13]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [14]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [15]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-08-11 01:41:08,332: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-11 01:41:08,342: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-11 01:41:08,344: INFO: common: created directory at: artifacts]
Found 3732 images belonging to 4 classes.
234/234 [==============================] - 242s 1s/step - loss: 4.6147 - accuracy: 0.6407
[2025-08-11 01:45:11,760: INFO: common: json file saved at: scores.json]


2025/08/11 01:45:13 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-08-11 01:45:14,749: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\anssa\AppData\Local\Temp\tmp_an1s2jy\model\data\model\assets
[2025-08-11 01:45:15,356: INFO: builder_impl: Assets written to: C:\Users\anssa\AppData\Local\Temp\tmp_an1s2jy\model\data\model\assets]


c:\Kidney_Disease_Classification_DL\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2025/08/11 01:46:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


In [21]:
from mlflow.tracking import MlflowClient

mlflow_uri = "https://dagshub.com/G-Sahil123/Kidney_Disease_Classification_DL.mlflow"
client = MlflowClient(tracking_uri=mlflow_uri)

client.transition_model_version_stage(
    name="VGG16Model",
    version=1,
    stage="Production",        
    archive_existing_versions=True                   
)

<ModelVersion: creation_timestamp=1754856980897, current_stage='Production', description='', last_updated_timestamp=1754861545374, name='VGG16Model', run_id='4656c096588b450e9eb6b08b7b61bf39', run_link='', source='mlflow-artifacts:/4dda71bf93b542d58adcb5c46909658e/4656c096588b450e9eb6b08b7b61bf39/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>